### Развертывание Spark

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.5.5/spark-3.5.5-bin-hadoop3.tgz
!tar xf spark-3.5.5-bin-hadoop3.tgz
!pip install -q findspark

In [ ]:
import os
from pyspark import SparkContext, SparkConf
from typing import NamedTuple
from datetime import datetime
from pyspark.sql.types import ArrayType, StringType
from pyspark.sql import functions as func
from pyspark.sql.window import Window
import numpy as np
import re
import findspark
from pyspark.sql import SparkSession

In [ ]:
findspark.init()

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.5-bin-hadoop3"

In [ ]:
spark = SparkSession.builder.appName("Сидоров 6404 ЛР2").config("spark.jars.packages", "com.databricks:spark-xml_2.12:0.14.0").getOrCreate()

In [ ]:
posts_path = os.path.join('posts_sample.xml')
languages_path = os.path.join('programming-languages.csv')

### Берем 10% данных

In [ ]:
posts_data = spark.read.format('xml').options(rowTag='row').load(posts_path)
posts_part = posts_data.sample(0.1)
posts_part.printSchema()


root
 |-- _AcceptedAnswerId: long (nullable = true)
 |-- _AnswerCount: long (nullable = true)
 |-- _Body: string (nullable = true)
 |-- _ClosedDate: timestamp (nullable = true)
 |-- _CommentCount: long (nullable = true)
 |-- _CommunityOwnedDate: timestamp (nullable = true)
 |-- _CreationDate: timestamp (nullable = true)
 |-- _FavoriteCount: long (nullable = true)
 |-- _Id: long (nullable = true)
 |-- _LastActivityDate: timestamp (nullable = true)
 |-- _LastEditDate: timestamp (nullable = true)
 |-- _LastEditorDisplayName: string (nullable = true)
 |-- _LastEditorUserId: long (nullable = true)
 |-- _OwnerDisplayName: string (nullable = true)
 |-- _OwnerUserId: long (nullable = true)
 |-- _ParentId: long (nullable = true)
 |-- _PostTypeId: long (nullable = true)
 |-- _Score: long (nullable = true)
 |-- _Tags: string (nullable = true)
 |-- _Title: string (nullable = true)
 |-- _ViewCount: long (nullable = true)



In [ ]:
languages_data = spark.read.option("header", True).option("inferSchema", True).option("timestampFormat", 'M/d/y H:m').csv(languages_path)
languages_data.printSchema()

root
 |-- name: string (nullable = true)
 |-- wikipedia_url: string (nullable = true)



### Выполнение задания

In [ ]:
language_names = [row.name.lower() for row in languages_data.select("name").collect()]

# Пользовательская функция получения ЯП из тегов
def get_languages(tag_string):
    if not tag_string or not isinstance(tag_string, str):
      return []

    found_tags = list(set(re.findall(r'<([^<>]+)>', tag_string)))
    found_tags_lower = [tag.lower().strip() for tag in found_tags]

    language_set = set(language_names)
    return [tag for tag in found_tags_lower if tag in language_set]

In [ ]:
get_languages_udf = func.udf(get_languages, ArrayType(StringType()))

# Добавляем к данным столбцы ЯП и года
processed_posts = (posts_part
                  .withColumn("languages", get_languages_udf(posts_part._Tags))
                  .withColumn("post_year", func.year(posts_part._CreationDate))
                  .select("post_year", func.explode("languages").alias("language"), "_ViewCount"))
processed_posts.printSchema()

root
 |-- post_year: integer (nullable = true)
 |-- language: string (nullable = true)
 |-- _ViewCount: long (nullable = true)



In [ ]:
# Отсеиваем по годам
filtered_posts = processed_posts.filter((processed_posts.post_year.between(2010, 2020)))

# Группируем по году и ЯПу
language_popularity = (filtered_posts
                      .groupBy("post_year", "language")
                      .agg(func.sum("_ViewCount").alias("total_views")))
language_popularity.printSchema()

root
 |-- post_year: integer (nullable = true)
 |-- language: string (nullable = true)
 |-- total_views: long (nullable = true)



In [ ]:
ranking_spec = Window.partitionBy('post_year').orderBy(func.desc('total_views'))
ranked_languages = language_popularity.withColumn('rank', func.dense_rank().over(ranking_spec))
ranked_languages.printSchema()

root
 |-- post_year: integer (nullable = true)
 |-- language: string (nullable = true)
 |-- total_views: long (nullable = true)
 |-- rank: integer (nullable = false)



In [ ]:
top = ranked_languages.where(ranked_languages['rank'] <= 10)
top.write.mode('overwrite').format('parquet').save('top.parquet')

saved_results = spark.read.parquet('top.parquet')
for year in range(2010, 2021):
    print(f"\n=== ТОП-10 языков программирования за {year} год ===")
    (saved_results
        .filter(saved_results.post_year == year)
        .orderBy("rank")
        .show(truncate=False))


=== ТОП-10 языков программирования за 2010 год ===
+---------+-----------+-----------+----+
|post_year|language   |total_views|rank|
+---------+-----------+-----------+----+
|2010     |java       |251465     |1   |
|2010     |ruby       |49635      |2   |
|2010     |applescript|30843      |3   |
|2010     |objective-c|12440      |4   |
|2010     |php        |10532      |5   |
|2010     |python     |7957       |6   |
|2010     |r          |3588       |7   |
|2010     |scala      |3330       |8   |
|2010     |haskell    |3000       |9   |
|2010     |perl       |2234       |10  |
+---------+-----------+-----------+----+


=== ТОП-10 языков программирования за 2011 год ===
+---------+----------+-----------+----+
|post_year|language  |total_views|rank|
+---------+----------+-----------+----+
|2011     |java      |95375      |1   |
|2011     |php       |95033      |2   |
|2011     |bash      |29372      |3   |
|2011     |javascript|24146      |4   |
|2011     |python    |12987      |5   |
|

698
